In [21]:
import random
from datetime import datetime, timedelta
import networkx as nx
from pyvis.network import Network
from collections import deque
import community
import psycopg2
import pandas as pd
import numpy as np
from collections import defaultdict

### 예시 데이터 만들기

In [3]:
transactions = []
names = [
    '신소율', '홍수아', '홍지호', '신민지', '신도윤', '한민준', '안현우', '최하은', '권현준', '홍지은',
    '윤지원', '김태윤', '이주원', '박서준', '최예은', '강지호', '배수아', '정하늘', '오유진', '서민재',
    '장하준', '민서연', '김다은', '송지후', '이하린', '윤지우', '한지호', '최다은', '정민준', '서지우'
]

start_time = datetime(2024, 1, 1, 9, 0, 0)

for _ in range(50):
    sender = random.choice(names)
    receiver = random.choice(names)

    # 출금인과 송금인이 같으면 재선택
    while sender == receiver:
        receiver = random.choice(names)
    
    amount = random.randint(10000, 5000000)
    time_delta = timedelta(minutes=random.randint(0, 60*24*30))
    transaction_time = (start_time + time_delta).strftime('%m-%d %H:%M')

    transactions.append((sender, receiver, amount, transaction_time))

In [4]:
transactions

[('배수아', '장하준', 891948, '01-29 07:30'),
 ('윤지원', '정민준', 3619976, '01-15 23:15'),
 ('윤지우', '박서준', 849241, '01-18 14:36'),
 ('안현우', '최다은', 4704594, '01-20 07:11'),
 ('최예은', '박서준', 3065978, '01-18 05:06'),
 ('한민준', '한지호', 2769197, '01-11 23:50'),
 ('정민준', '신도윤', 2854373, '01-03 22:19'),
 ('최다은', '신도윤', 4849216, '01-09 06:47'),
 ('배수아', '장하준', 4609021, '01-07 00:24'),
 ('홍지호', '김다은', 3613329, '01-14 05:52'),
 ('서지우', '신소율', 3728972, '01-09 04:37'),
 ('정하늘', '이주원', 1759601, '01-14 01:16'),
 ('송지후', '박서준', 2783175, '01-14 05:54'),
 ('이주원', '홍지호', 817667, '01-14 11:32'),
 ('배수아', '이주원', 219659, '01-14 04:45'),
 ('한민준', '안현우', 295623, '01-30 17:35'),
 ('최하은', '최다은', 2059096, '01-12 12:59'),
 ('정민준', '권현준', 3509211, '01-05 13:46'),
 ('정하늘', '배수아', 3977163, '01-29 17:19'),
 ('이하린', '한민준', 2006557, '01-18 19:38'),
 ('윤지우', '민서연', 1185671, '01-11 20:43'),
 ('김태윤', '강지호', 3978095, '01-05 14:17'),
 ('한지호', '홍지호', 965472, '01-05 20:38'),
 ('서민재', '서지우', 776658, '01-27 22:05'),
 ('정하늘', '윤지우', 1349434

### DB에서 데이터 가져오기

In [58]:
# DB 연결
conn = psycopg2.connect(
    host="192.168.0.69",
    port=10360,
    dbname="ysmartdata",
    user="postgres",
    password="ShbcdDevelopPostgres!!@#)"
)

query = """
SELECT * FROM manager.linked_graph
WHERE a->'account'->'properties'->>'name' = '구주환'
"""

df = pd.read_sql(query, conn)

# 연결 종료
conn.close()

C:\Users\yh\AppData\Local\Temp\ipykernel_7484\413194769.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [59]:
transaction_list = []

for i in range(len(df)):
    current_row = df.iloc[i]
    
    r1 = current_row['r1']
    r2 = current_row['r2']
    r3 = current_row['r3']

    start_date = current_row['start_date']
    end_date = current_row['end_date']

    if r1:
        a_name = current_row['a']['account']['properties']['name']
        b_name = current_row['b']['account']['properties']['name']
        r1_ipzygubun = current_row['r1']['transact']['properties']['ipzygubun']
        aggr_val1 = current_row['aggr_val1'].split(',')
        trans_count1 = aggr_val1[0]
        avg_trans_amount1 = aggr_val1[1]
        total_trans_amount1 = aggr_val1[2]

        if r1_ipzygubun == 2: # 출금이면
            transaction_list.append((a_name, b_name, total_trans_amount1, start_date))
        else: # 입금이면
            transaction_list.append((b_name, a_name, total_trans_amount1, start_date))

    
    if r2:
        c_name = current_row['c']['account']['properties']['name']
        r2_ipzygubun = current_row['r2']['transact']['properties']['ipzygubun']
        aggr_val2 = current_row['aggr_val2'].split(',')
        trans_count2 = aggr_val2[0]
        avg_trans_amount2 = aggr_val2[1]
        total_trans_amount2 = aggr_val2[2]

        if r2_ipzygubun == 2: # 출금이면
            transaction_list.append((b_name, c_name, total_trans_amount2, start_date))
        else: # 입금이면
            transaction_list.append((c_name, b_name, total_trans_amount2, start_date))
    
    if r3:
        d_name = current_row['d']['account']['properties']['name']
        r3_ipzygubun = current_row['r3']['transact']['properties']['ipzygubun']
        aggr_val3 = current_row['aggr_val3'].split(',')
        trans_count3 = aggr_val3[0]
        avg_trans_amount3 = aggr_val3[1]
        total_trans_amount3 = aggr_val3[2]

        if r3_ipzygubun == 2: # 출금이면
            transaction_list.append((c_name, d_name, total_trans_amount3, start_date))
        else: # 입금이면
            transaction_list.append((d_name, c_name, total_trans_amount3, start_date))

In [64]:
transaction_list

[('구준희', '구주환', '4000', '2025-03-06'),
 ('강지우', '구준희', '4000', '2025-03-06'),
 ('심윤채', '강지우', '18000', '2025-03-06'),
 ('윤은율', '구주환', '2000', '2025-04-03'),
 ('임이안', '윤은율', '4000', '2025-04-03'),
 ('성세아', '임이안', '6000', '2025-04-03'),
 ('윤은율', '구주환', '2000', '2025-03-04'),
 ('임이안', '윤은율', '4000', '2025-03-04'),
 ('주시진', '임이안', '12000', '2025-03-04'),
 ('윤은율', '구주환', '2000', '2025-03-15'),
 ('임이안', '윤은율', '4000', '2025-03-15'),
 ('우준혁', '임이안', '18000', '2025-03-15'),
 ('윤은율', '구주환', '2000', '2025-01-01'),
 ('임이안', '윤은율', '4000', '2025-01-01'),
 ('권유운', '임이안', '24000', '2025-01-01'),
 ('윤은율', '구주환', '2000', '2025-01-24'),
 ('임이안', '윤은율', '4000', '2025-01-24'),
 ('하예안', '임이안', '30000', '2025-01-24'),
 ('윤은율', '구주환', '2000', '2025-03-10'),
 ('임이안', '윤은율', '4000', '2025-03-10'),
 ('손예우', '임이안', '36000', '2025-03-10'),
 ('윤은율', '구주환', '2000', '2025-04-26'),
 ('임이안', '윤은율', '4000', '2025-04-26'),
 ('류선우', '임이안', '42000', '2025-04-26'),
 ('윤은율', '구주환', '2000', '2025-03-18'),
 ('임이안', '윤은율', '4

### 전체 그래프 그리기

In [69]:
transaction_dict = defaultdict(list)

for sender, receiver, amount, time in transaction_list:
    transaction_dict[(sender, receiver)].append((amount, time))

transaction_dict.keys()

dict_keys([('구준희', '구주환'), ('강지우', '구준희'), ('심윤채', '강지우'), ('윤은율', '구주환'), ('임이안', '윤은율'), ('성세아', '임이안'), ('주시진', '임이안'), ('우준혁', '임이안'), ('권유운', '임이안'), ('하예안', '임이안'), ('손예우', '임이안'), ('류선우', '임이안'), ('윤서찬', '임이안'), ('심다율', '임이안'), ('이민주', '임이안'), ('차지수', '윤은율'), ('나유운', '차지수'), ('강예원', '차지수'), ('심도영', '차지수'), ('노하원', '차지수'), ('정시하', '차지수'), ('심민주', '차지수'), ('류민서', '차지수'), ('신승훈', '차지수'), ('우민겸', '차지수'), ('신현성', '차지수'), ('이서현', '윤은율'), ('민혜지', '이서현'), ('원수영', '이서현'), ('류은율', '이서현'), ('윤현지', '이서현'), ('류예린', '이서현'), ('주여진', '이서현'), ('남예솔', '이서현'), ('임도영', '이서현'), ('유다은', '이서현'), ('류병현', '이서현'), ('장연우', '윤은율'), ('구다경', '장연우'), ('최다나', '장연우'), ('박아린', '장연우'), ('한하유', '장연우'), ('한서연', '장연우'), ('채주은', '장연우'), ('나예지', '장연우'), ('천유정', '장연우'), ('유예경', '구준희'), ('장수연', '유예경'), ('안정아', '유예경'), ('임윤서', '유예경'), ('박연호', '유예경'), ('김은영', '구준희'), ('손은비', '김은영'), ('차효린', '김은영'), ('민서우', '김은영'), ('진지환', '김은영'), ('차유찬', '김은영'), ('윤유우', '김은영'), ('강민겸', '김은영'), ('강정아', '김은영'), ('서은서', '김은영'), ('윤시하', '김은영'

In [77]:
# 전체 그래프 그리기
def draw_full_graph(transactions, output_path):
    net = Network(height='800px', width='100%', directed=True)

    transaction_dict = defaultdict(list)

    for sender, receiver, amount, time in transactions:
        transaction_dict[(sender, receiver)].append((amount, time))

    for sender, receiver in transaction_dict.keys():

        net.add_node(sender, label=sender)
        net.add_node(receiver, label=receiver)

        total_amount = sum([int(float(amount)) for amount, _ in transaction_dict[(sender, receiver)]])
        title = f"총 거래금액 : {total_amount}원"

        net.add_edge(sender, receiver, title=title, value=total_amount)

    net.save_graph(output_path)

In [75]:
draw_full_graph(transaction_list, 'db_full_graph.html')

In [57]:
# 부분 그래프 그리기
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

# BFS (너비 우선 탐색)
def find_edges(G, start_node, max_depth):
    visited = set()
    visited.add(start_node) # 시작 노드 방문 추가
    queue = deque()
    queue.append((start_node, 0))
    result_edges = set()

    while queue:
        node, depth = queue.popleft()
        if depth >= max_depth:
            continue
        # 출금 (내가 돈을 보낸 사람)
        for neighbor in G.successors(node): 
            result_edges.add((node, neighbor))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
        # 입금 (나에게 돈을 보낸 사람)
        for neighbor in G.predecessors(node):
            result_edges.add((neighbor, node))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
    return list(result_edges)

start_node = '김태윤'
max_depth = 2
sub_edges = find_edges(G, start_node, max_depth)

# pyvis 시각화
net = Network(height='800px', width='100%', directed=True)
added_nodes = set()
for sender, receiver in sub_edges:
    for node in (sender, receiver):
        if node not in added_nodes:
            net.add_node(node, lable=node)
            added_nodes.add(node)
    edge_data = G.get_edge_data(sender, receiver)
    if edge_data: # edge_data가 있으면
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])
    else:
        # edge_data가 없을 수도 있으므로 예외 처리
        net.add_edge(sender, receiver)

# 중심 노드 강조
net.get_node(start_node)['color'] = 'red'

# 저장
output_path = 'graph.html'
net.save_graph(output_path)
output_path

'graph.html'

In [15]:
def draw_partial_graph(transactions, start_node_name,
                       max_depth, output_path):
    # 부분 그래프 그리기
    G = nx.DiGraph()
    for sender, receiver, amount, time in transactions:
        G.add_edge(sender, receiver, amount=amount, time=time)

    # BFS (너비 우선 탐색)
    def find_edges(G, start_node, max_depth):
        visited = set()
        visited.add(start_node) # 시작 노드 방문 추가
        queue = deque()
        queue.append((start_node, 0))
        result_edges = set()

        while queue:
            node, depth = queue.popleft()
            if depth >= max_depth:
                continue
            # 출금 (내가 돈을 보낸 사람)
            for neighbor in G.successors(node): 
                result_edges.add((node, neighbor))
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, depth+1))
            # 입금 (나에게 돈을 보낸 사람)
            for neighbor in G.predecessors(node):
                result_edges.add((neighbor, node))
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, depth+1))
        return list(result_edges)

    max_depth = max_depth
    sub_edges = find_edges(G, start_node_name, max_depth)

    # pyvis 시각화
    net = Network(height='800px', width='100%', directed=True)
    added_nodes = set()
    for sender, receiver in sub_edges:
        for node in (sender, receiver):
            if node not in added_nodes:
                net.add_node(node, lable=node)
                added_nodes.add(node)
        edge_data = G.get_edge_data(sender, receiver)
        if edge_data: # edge_data가 있으면
            net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])
        else:
            # edge_data가 없을 수도 있으므로 예외 처리
            net.add_edge(sender, receiver)

    # 중심 노드 강조
    net.get_node(start_node_name)['color'] = 'red'

    # 저장
    net.save_graph(output_path)
    output_path

In [16]:
draw_partial_graph(transaction_list[:1000], '구주환', 2, 'partial_graph.html')

### 커뮤니티 디텍션

In [91]:
# Louvain, Leidan같은 커뮤니티 디텍션 알고리즘들은
# 방향성 없는 그래프를 기본으로 가정
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

G_undirected = G.to_undirected()

# 커뮤니티 검출
partition = community.best_partition(G_undirected)

# 결과
partition

{'권현준': 2,
 '안현우': 3,
 '김다은': 2,
 '배수아': 2,
 '홍지은': 4,
 '박서준': 4,
 '한지호': 4,
 '민서연': 4,
 '홍지호': 1,
 '서지우': 1,
 '김태윤': 1,
 '이주원': 3,
 '윤지우': 3,
 '신도윤': 2,
 '홍수아': 0,
 '서민재': 0,
 '윤지원': 4,
 '최예은': 2,
 '장하준': 2,
 '오유진': 2,
 '최다은': 2,
 '최하은': 3,
 '신민지': 2,
 '신소율': 3,
 '이하린': 4,
 '강지호': 0,
 '정민준': 0,
 '정하늘': 4,
 '송지후': 0,
 '한민준': 3}

### 시각화에 커뮤니티 반영

In [ ]:
color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

net = Network(height='800px', width='100%', directed=True)

for node in G.nodes():
    comm_id = partition.get(node, 0) # 커뮤니티 id 추출
    color = color_list[comm_id % len(color_list)]
    net.add_node(node, label=node, color=color)

for sender, receiver in G.edges():
    edge_data = G.get_edge_data(sender, receiver)
    net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])

net.save_graph('community2.html')

### 특정 커뮤니티만 시각화

In [99]:
comm_number = 0
selected_community = []

color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

for node, comm_id in partition.items():
    if comm_id == comm_number:
        selected_community.append(node)

net = Network(height='800px', width='100%', directed=True)

# 노드 먼저 추가
for node in selected_community:
    net.add_node(node, label=node, color=color_list[comm_number % len(color_list)])

for sender, receiver in G.edges():
    if (sender in selected_community) and (receiver in selected_community):
        edge_data = G.get_edge_data(sender, receiver)
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])

net.save_graph('selected_community.html')


In [83]:
def draw_community_detection_partial_graph(transactions, start_node_name, max_depth, output_path):
    # 전체 그래프 생성
    G = nx.DiGraph()
    for sender, receiver, amount, time in transactions:
        G.add_edge(sender, receiver, amount=int(float(amount)), time=time)

    # BFS 기반 부분 그래프 탐색
    def find_edges(G, start_node, max_depth):
        visited = set()
        visited.add(start_node) # 시작 노드 방문 추가
        queue = deque()
        queue.append((start_node, 0))
        result_edges = set()

        while queue:
            node, depth = queue.popleft()
            if depth >= max_depth:
                continue
            # 출금 (내가 돈을 보낸 사람)
            for neighbor in G.successors(node): 
                result_edges.add((node, neighbor))
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, depth+1))
            # 입금 (나에게 돈을 보낸 사람)
            for neighbor in G.predecessors(node):
                result_edges.add((neighbor, node))
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, depth+1))
        return list(result_edges)

    # 부분 그래프 edge 추출 및 서브그래프 생성
    sub_edges = find_edges(G, start_node_name, max_depth)
    sub_nodes = set()
    for u, v in sub_edges:
        sub_nodes.add(u)
        sub_nodes.add(v)
    subgraph = G.subgraph(sub_nodes).copy()

    # Louvain 커뮤니티 탐지 (undirected 필요)
    undirected_subgraph = subgraph.to_undirected()
    partition = community.best_partition(undirected_subgraph)

    color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

    net = Network(height='800px', width='100%', directed=True)

    for node in subgraph.nodes():
        comm_id = partition.get(node, 0) # 커뮤니티 id 추출
        color = color_list[comm_id % len(color_list)]
        net.add_node(node, label=node, color=color)

    # for sender, receiver in subgraph.edges():
    #     edge_data = subgraph.get_edge_data(sender, receiver)
    #     print(edge_data)
    #     net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])

    edge_dict = defaultdict(list)

    for sender, receiver, amount, time in transaction_list:
        edge_dict[(sender, receiver)].append((int(float(amount)), time))

    for sender, receiver in subgraph.edges():
        total_amount = sum([amount for amount, _ in edge_dict[(sender, receiver)]])
        
        amount_time_list = [f"{amount}원 {time}" for amount, time in edge_dict[(sender, receiver)]]

        title = f"총 거래금액 : {total_amount}원\n{'\n'.join(amount_time_list)}"

        net.add_edge(sender, receiver, title=f"{title}", value=total_amount)

    net.save_graph(output_path)

In [84]:
draw_community_detection_partial_graph(transaction_list, '구주환', 3, 'community3.html')